# Class for extraction predicates and arguments (noun phrases)

In [64]:
from isanlp_srl_framebank.pipeline_default import PipelineDefault  

class TextGroups(object):
    HOST = 'HOST'
    MORPH_PORT = 3333
    SYNTAX_PORT = 3334
    SRL_PORT = 3335
    
    
    GROUPS_PARAMETERS = {
        'predicate': {
            'TYPES_HEAD': ['VERB'],
            'TYPES_ALL': ['ADJ', 'PART', 'CCONJ', 'ADV', 'ADP'],
            'ENTITY_LEMMAS': [
                u'не', u'ни'
            ]
        },
        'argument' : {
            'TYPES_HEAD': ['NOUN','ADJ', 'CCONJ', 'PRON', 'ADP'],
            'TYPES_ALL': ['NOUN','ADJ', 'ADV', 'CCONJ', 'PRON', 'ADP'],
            'ENTITY_LEMMAS': []
        }
    }
    
    
    def __init__(self, analyzed_text):
        self.pipeline = PipelineDefault(address_morph=(self.HOST, self.MORPH_PORT),
                          address_syntax=(self.HOST, self.SYNTAX_PORT),
                          address_srl=(self.HOST, self.SRL_PORT))
        self.analyzed_text = analyzed_text
        self.__analyze_text()
        self.children_annotation = self.__extract_children_annotation()
        
        self.groups = self.__extract_goups()
        
    def __analyze_text(self):
        res = self.pipeline(self.analyzed_text)
        self.syntax = res['syntax_dep_tree']
        self.morph = res['morph']
        self.lemma = res['lemma']
        self.tokens = res['tokens']
        self.sentences = res['sentences']

    
    def __extract_children_annotation(self):
        children_annotation = []
        for sent_pos, sent in enumerate(self.syntax):
            sent_children_annotation = [[] for i in range(len(sent))]
            for word_pos, word in enumerate(sent):
                if word.parent != -1:
                    sent_children_annotation[word.parent].append(word_pos)
            children_annotation.append(sent_children_annotation)
        return children_annotation
    
    
    
    def __extract_synt_top(self, synt_sent):
        for word_pos, synt_word in enumerate(synt_sent):
            if synt_word.link_name == 'ROOT':
                return word_pos

        
        
    def __extract_phrases(self, extract_type, sent_pos, current_word_pos, is_top = True, is_father_in_np = False):
        types_head = self.GROUPS_PARAMETERS[extract_type]['TYPES_HEAD']
        types_all = self.GROUPS_PARAMETERS[extract_type]['TYPES_ALL']
        entity_lemmas = self.GROUPS_PARAMETERS[extract_type]['ENTITY_LEMMAS']
        
        sent_morph = self.morph[sent_pos]
        phrase = []
        phrases = []
        current_word_children = self.children_annotation[sent_pos][current_word_pos]

        checking_phrase_types = None
        checking_lemmas = []
        if is_father_in_np:
            checking_phrase_types = types_all
            checking_lemmas = entity_lemmas
        else:
            checking_phrase_types = types_head

        if (self.morph[sent_pos][current_word_pos].get('fPOS') in checking_phrase_types) or (self.lemma[sent_pos][current_word_pos] in checking_lemmas):
            phrase.append(current_word_pos)
            for word_pos in current_word_children:
                new_phrase, new_phrases = self.__extract_phrases(extract_type, sent_pos, word_pos, False, True)
                phrase += new_phrase
                phrases += new_phrases
        else:
            for word_pos in current_word_children:
                new_phrase, new_phrases = self.__extract_phrases(extract_type, sent_pos, word_pos, False)
                phrases += new_phrases ## NEW!!
                if len(new_phrase) > 0:
                    phrases.append(new_phrase)
        if is_top:
            if len(phrase) > 0:
                phrases.append(phrase)
            return phrases
        else:
            return phrase, phrases

        
        
        
        
    def __extract_goups(self):
        groups = []
        
        for sent_pos, synt_sent in enumerate(self.syntax):
            sent_top = self.__extract_synt_top(synt_sent)
            predicates = self.__extract_phrases('predicate', sent_pos, sent_top)
            for predicate in predicates:
                
                predicate_children = [child_idx for word_idx in predicate
                                      for child_idx in self.children_annotation[sent_pos][word_idx]
                                      if not child_idx in predicate]
                predicate_children = list(set(sorted(predicate_children)))
                arguments = []
                for predicate_child in predicate_children:
                    arguments += self.__extract_phrases('argument', sent_pos, predicate_child)
            
                groups.append({
                    'sent_pos' : sent_pos,
                    'arguments': [sorted(arg) for arg in arguments],
                    'predicate' : sorted(predicate)
                })
        return groups
            
    

    def get_pretty_phrase(self, sent_pos, phrase):
        sentence_offsets = self.sentences[sent_pos]
        sentence_tokens = self.tokens[sentence_offsets.begin:sentence_offsets.end]
        return ' '.join([sentence_tokens[word_pos].text for word_pos in phrase])
   


    def get_pretty(self):
        pretty_groups = []
        for group in self.groups:  
            pretty_groups.append({
                'predicate' : self.get_pretty_phrase(group['sent_pos'], group['predicate']),
                'arguments' : [self.get_pretty_phrase(group['sent_pos'], arg) for arg in group['arguments']]
            })
        return pretty_groups


groups = TextGroups(text)

# Main 

In [62]:
text = None
with open('example_text.txt', 'r') as rfile:
    text = rfile.read()
groups = TextGroups(text)
pretty_groups = groups.get_pretty()

for group in pretty_groups:
    print('___________________')
    print(group['predicate'])
    print()
    for arg in group['arguments']:
        print(arg)

    print()
    print()


___________________
подписал

Мексиканский боксер Сауль Альварес
контракт со стриминговым сервисом


___________________
имеющейся



___________________
выступающий

в среднем весе
поясами чемпиона мира по версиям


___________________
рассчитанному

до года


___________________
получит

По информации
минимум
до года
за боев по соглашению
в среднем весе
поясами чемпиона мира по версиям
летний спортсмен


___________________
подписал

с
Майами
контракт
который
Марлинс на лет
в году


___________________
стал рекордным

Контракт мексиканца
достижение бейсболиста
Нью
в истории спорта
который
в году
с
Майами
контракт
Марлинс на лет
Йорк Янкис Джанкарло Стэнтона


___________________
проведет

с
Альварес
Первый бой в рамках соглашения
декабря


___________________
встретится

во втором среднем весе
В поединке за титул
с британцем Рокки Филдингом
он


___________________
пройдет в Нью

Бой
на
Йорке
Мэдисон Сквер Гарден


___________________
Канело станет

Для
поединок
м в карьере


_______